# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the (small) GPT-2 model (120mil parameters).



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [1]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [2]:
cfg = breaching.get_config(overrides=["attack=decepticon", "case=10_causal_lang_training", 
                                     "case/server=malicious-transformer"])
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

Investigating use case causal_lang_training with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [3]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [32] # This is the sequence length

cfg.case.server.provide_public_buffers = True # Send server signal to disable dropout
cfg.case.server.has_external_data = True  # Not strictly necessary, but could also use random text (see Appendix)
cfg.case.data.tokenizer = "gpt2"
cfg.case.model = "transformerS" # Could also choose "gpt2S" which contains ReLU activations
cfg.case.server.pretrained = False

## Attack hyperparameters:

# Server side:
cfg.case.server.param_modification.v_length = 8 # Length of the sentence component
cfg.case.server.param_modification.eps = 1
cfg.case.server.param_modification.measurement_scale=1  # 1e12 # Circumvent GELU
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 1e8
cfg.case.server.param_modification.sequence_token_weight = 1


# Attacker side:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 
cfg.attack.token_strategy="embedding-norm" # no decoder bias in GPT
cfg.attack.embedding_token_weight=0.0 # Setting e.g. 0.25 here can improve performance slightly for long sequences
cfg.attack.separation = "subtraction"#"decorrelation"

cfg.attack.breach_reduction = "total-weight"
cfg.attack.undivided=True
## DP

cfg.case.user.local_diff_privacy.input_noise = 0.0  # input up to 1.0 is ok
cfg.case.user.local_diff_privacy.gradient_noise =1e-8
cfg.case.user.local_diff_privacy.per_example_clipping = 0
cfg.case.user.local_diff_privacy.distribution = 'gaussian'

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture transformerS loaded with 53,091,409 parameters and 2,560,000 buffers.
Overall this is a data ratio of 1659106:1 for target shape [1, 32] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 1

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    Defense: Local gaussian gradient noise with strength 9.99999993922529e-09.
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specifi

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)  

Found attention of shape torch.Size([1536, 512]).
Computing feature distribution before the probe layer Linear(in_features=512, out_features=512, bias=True) from external data.
Feature mean is -0.03757932037115097, feature std is 0.9479761719703674.
Computing user update on user 1 in model mode: eval.


In [6]:
['transformer_encoder.layers.0.self_attn.in_proj_weight:0.0',
 'transformer_encoder.layers.0.self_attn.in_proj_bias:0.0',
 'transformer_encoder.layers.0.self_attn.out_proj.weight:0.8733341097831726',
 'transformer_encoder.layers.0.self_attn.out_proj.bias:0.21867378056049347',
 'transformer_encoder.layers.0.linear1.weight:0.00801785197108984',
 'transformer_encoder.layers.0.linear1.bias:0.00039870699401944876',
 'transformer_encoder.layers.0.linear2.weight:6.721737384796143',
 'transformer_encoder.layers.0.linear2.bias:0.3053097128868103',
 'transformer_encoder.layers.0.norm1.weight:0.2597469091415405',
 'transformer_encoder.layers.0.norm1.bias:0.3053097128868103',
 'transformer_encoder.layers.0.norm2.weight:0.2685771584510803',
 'transformer_encoder.layers.0.norm2.bias:0.3059074580669403',
 'encoder.weight:3.918757915496826',
 'decoder.weight:4.312182426452637'];

In [7]:
[f"{n}:{g.norm().item()}" for (n, p), g in zip(user.model.named_parameters(), shared_data["gradients"])][:14]

['transformer_encoder.layers.0.self_attn.in_proj_weight:8.88554859557189e-06',
 'transformer_encoder.layers.0.self_attn.in_proj_bias:3.814743081420602e-07',
 'transformer_encoder.layers.0.self_attn.out_proj.weight:0.5064964294433594',
 'transformer_encoder.layers.0.self_attn.out_proj.bias:0.20190459489822388',
 'transformer_encoder.layers.0.linear1.weight:0.31360772252082825',
 'transformer_encoder.layers.0.linear1.bias:0.00013857729209121317',
 'transformer_encoder.layers.0.linear2.weight:6.309071063995361',
 'transformer_encoder.layers.0.linear2.bias:0.2870209813117981',
 'transformer_encoder.layers.0.norm1.weight:0.2538856565952301',
 'transformer_encoder.layers.0.norm1.bias:0.2870004177093506',
 'transformer_encoder.layers.0.norm2.weight:0.2675231993198395',
 'transformer_encoder.layers.0.norm2.bias:0.28992146253585815',
 'encoder.weight:3.869091749191284',
 'decoder.weight:4.290277481079102']

In [8]:
user.print(true_user_data)

 The Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, is a building located in MacArthur Park in downtown Little Rock, Arkansas


### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [9]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Proceeded to cut estimated token distribution at 1.50.
Recovered tokens tensor([   13,    13,    50,   257,   262,   286,   287,   287,   318,   318,
          355,   383,   471,   635,   837,   837,  1900,  2615,  3250,  4631,
         4631,  5140,  7703,  7703,  7703,  8765,  9436, 11819, 11819, 13837,
        13837, 46626]) through strategy embedding-norm.
Recovered 512 embeddings with positional data from imprinted layer.
tensor([6.4407e-04, 6.0268e-04, 5.6434e-04, 5.5086e-04, 5.4691e-04, 2.3977e-04,
        2.2925e-04, 1.1306e-04, 1.1056e-04, 1.1686e-05, 2.9096e-06, 1.9644e-09,
        1.8579e-09, 1.5881e-09, 1.5716e-09, 1.5319e-09, 1.4875e-09, 1.4527e-09,
        1.3173e-09, 1.2234e-09, 1.2016e-09, 1.1956e-09, 1.1621e-09, 1.1506e-09,
        1.1063e-09, 1.1015e-09, 1.0939e-09, 1.0896e-09, 1.0713e-09, 1.0702e-09,
        1.0673e-09, 1.0657e-09, 1.0501e-09, 9.9716e-10, 9.9595e-10, 9.9374e-10,
        9.7673e-10, 9.7430e-10, 9.6475e-10, 9.6247e-10, 9.4863e-10, 9.2029e-10,
        9.

Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [10]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

METRICS: | Accuracy: 0.1250 | S-BLEU: 0.06 | FMSE: 2.6643e+00 | 
 G-BLEU: 0.25 | ROUGE1: 0.89| ROUGE2: 0.08 | ROUGE-L: 0.37| Token Acc: 93.75% | Label Acc: 93.75%


And finally, we also plot the reconstructed data:

In [11]:
user.print(reconstructed_user_data)

 as Tower Building Rock Little located known Little Park also building Little MacArthur Rock The a Arsenal Arsenal is of the. US in in,, is downtown Building.


### Notes:
* There are a variety of hyperparameters to the attack which are set to reasonable defaults. Performance of the attack could be improved in some unusual use cases (datasets or models) by tuning these parameters further.
* In this example, dropout is disabled under the assumption that this is a parameter that can be controlled in the server update. The optimal attack simply disables dropout. However, the attack can still succeed when dropout is enforced by the user, albeit with a minor loss in reconstruction quality.
* This example also assumes complete freedom to choose the parameter vector, for this reason we circumvent the smooth part of the GELU activation with a "very" large measurement vector magnitude. This is arguably excessive for only a small again in accuracy.
* We also want to re-emphasize that the design space of these parameter modification attacks is large. A defense against the specific parameter modification described here is unlikely to be safe in general!